In [49]:
import json
from pathlib import Path
import shutil

In [50]:
import cv2
from tqdm import tqdm

# Mirror left leaning poses to right leaning poses

In [51]:
angle_l_limit = 52  # Left leaning arbitrary limit

In [56]:
input_json_pose_file = '../data/filtered_pose.json'
input_json_pose_folder = '../data/wiki_crop_filtered_pose'

output_json_pose_mirrored_file = '../data/filtered_pose_mirrored.json'
output_json_pose_mirrored_folder = '../data/wiki_crop_filtered_mirrored_pose'

In [57]:
with open(input_json_pose_file, 'r') as f:
    records = json.load(f)

In [58]:
Path(output_json_pose_mirrored_folder).mkdir(exist_ok=True)

count = 0
for file_name in tqdm(records):
    values = records[file_name]
    
    if not values['angle_l'] > angle_l_limit:
    # if not values['angle_l'] > values['angle_r']:
        shutil.copy(f'{input_json_pose_folder}/{file_name}', f'{output_json_pose_mirrored_folder}/{file_name}')
        continue

    image = cv2.imread(f'{input_json_pose_folder}/{file_name}')
    _, width, _ = image.shape
    image = cv2.flip(image, 1)
    cv2.imwrite(f'{output_json_pose_mirrored_folder}/{file_name}', image)
    
    new_bbox = [
        width - values['bbox'][2], 
        values['bbox'][1], 
        width - values['bbox'][0], 
        values['bbox'][3]
    ]

    new_landmarks = {}
    for key, values in new_landmarks.items():
        new_landmarks[key] = [width - values[0], values[1]]
    
    # Flip the angles
    angle_l, angle_r = values['angle_r'], values['angle_l']
    
    values.update({
        'bbox': new_bbox,
        'landmarks': new_landmarks,
        'angle_l': angle_l,
        'angle_r': angle_r
    })
    
    count += 1

100%|██████████| 6661/6661 [00:12<00:00, 529.59it/s]


In [59]:
print(count)

2674


In [60]:
with open(output_json_pose_mirrored_file, 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)